In [1]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
import numpy as np
# fix random seed for reproducibility
np.random.seed(0)
nb_classes = 12

Using TensorFlow backend.


In [2]:
# data_processed.csv
# load pima indians dataset
df = pd.read_csv('data_processed.csv')
# Get testSet
df_test = df.sample(frac=0.2, random_state=47)
df_test.to_csv('testSet.csv', index=False)
# Subtract to get trainingSet
df_train = df[~df.index.isin(df_test.index)]
df_train.to_csv('trainingSet.csv', index=False)

In [3]:
trainingSet = pd.read_csv('trainingSet.csv')
testSet = pd.read_csv('testSet.csv')

In [4]:
Y_train_raw = np.array(trainingSet['Value'])
trainSet_X = trainingSet.drop('Value', axis=1)
X_train = np.array(trainSet_X)
Y_train  = np.zeros((len(trainSet_X), nb_classes))
print Y_train_raw
for i in range(len(Y_train_raw)):
    Y_train[i][Y_train_raw[i] - 1] = 1.0
print X_train.shape

[2 2 3 ... 5 2 1]
(4850, 921)


In [5]:
Y_test = np.array(testSet['Value'])
testSet_X = testSet.drop('Value', axis=1)
X_test = np.array(testSet_X)
Y_test = Y_test - [1]

In [6]:
# create model
nb_classes = 12
model = Sequential()
model.add(Dense(1024, input_dim=921, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

#model.add(Dense(2, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# print X_train
# print Y_train
# Fit the model
model.fit(X_train, Y_train, epochs=10, batch_size=10, shuffle=True, class_weight='auto',)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4850/4850 [==============================] - 7s 1ms/step - loss: 0.2292 - acc: 0.9160
Epoch 2/10
4850/4850 [==============================] - 4s 864us/step - loss: 0.2068 - acc: 0.9157
Epoch 3/10
4850/4850 [==============================] - 4s 904us/step - loss: 0.2016 - acc: 0.9177
Epoch 4/10
4850/4850 [==============================] - 4s 893us/step - loss: 0.2051 - acc: 0.9179
Epoch 5/10
4850/4850 [==============================] - 4s 858us/step - loss: 0.2010 - acc: 0.9193
Epoch 6/10
4850/4850 [==============================] - 4s 901us/step - loss: 0.1997 - acc: 0.9195
Epoch 7/10
4850/4850 [==============================] - 4s 907us/step - loss: 0.1969 - acc: 0.9195
Epoch 8/10
4850/4850 [==============================] - 5s 943us/step - loss: 0.1945 - acc: 0.9200
Epoch 9/10
4850/4850 [==============================] - 5s 940us/step - loss: 0.1921 - acc: 0.9211
Epoch 10/10
4850/4850 [==============================] - 5s 942

In [8]:
raw_predictions = model.predict(X_test, batch_size=10, verbose=0)
print raw_predictions
predicted = np.argmax(raw_predictions, axis=1)

[[4.0802178e-01 5.0484490e-01 7.7668406e-02 ... 3.0472850e-06
  1.2473454e-08 9.0863299e-09]
 [5.1655805e-01 4.4439891e-01 3.7374757e-02 ... 3.1577758e-07
  5.2386229e-10 3.8623940e-10]
 [4.8281559e-01 4.5982048e-01 5.3551659e-02 ... 1.4132298e-06
  4.0247055e-09 3.1226051e-09]
 ...
 [3.4185416e-01 5.3282464e-01 1.0460607e-01 ... 5.7007078e-06
  3.2374274e-08 2.2015145e-08]
 [4.8688957e-01 4.5751718e-01 5.2041162e-02 ... 1.2817227e-06
  3.5039112e-09 2.7165201e-09]
 [2.8498763e-01 5.3467780e-01 1.3681675e-01 ... 1.0815006e-05
  7.9519069e-08 5.5597710e-08]]


In [9]:
count = 0
print Y_test
for i in range(len(Y_test)):
    if predicted[i] == Y_test[i]:
        count += 1
print 1.0 * count/len(Y_test)

[3 0 0 ... 2 0 5]
0.329207920792


In [10]:
# print np.count_nonzero(Y_test)

#testSet['Value'].value_counts()
print testSet['Value'].value_counts()

1     387
2     309
3     150
4     107
6      96
5      75
7      35
8      28
9      12
11      6
10      6
12      1
Name: Value, dtype: int64


In [11]:
print Y_test

[3 0 0 ... 2 0 5]
